In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import os
import gc

In [2]:
# 设置使代码的可读性更高
sns.set(style='white',context='notebook',palette='deep')
sns.set_style('white')
pd.set_option('display.max_columns',500)

In [108]:
train_A = pd.read_csv(r'E:/数据分析数据集/DataTrain/DataTrain/A_train.csv')
train_B = pd.read_csv(r'E:/数据分析数据集/DataTrain/DataTrain/B_train.csv')
test = pd.read_csv(r'E:/数据分析数据集/DataTrain/DataTrain/B_test.csv')

In [7]:
train_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 491 entries, no to UserInfo_270
dtypes: float64(489), int64(2)
memory usage: 149.8 MB


In [8]:
train_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Columns: 491 entries, no to UserInfo_270
dtypes: float64(488), int64(3)
memory usage: 15.0 MB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13463 entries, 0 to 13462
Columns: 490 entries, no to UserInfo_270
dtypes: float64(488), int64(2)
memory usage: 50.3 MB


In [10]:
# 数据维度
print('shape of train_A:',train_A.shape)
print('shape of train_B:',train_B.shape)
print('shape of test:',test.shape)

shape of train_A: (40000, 491)
shape of train_B: (4000, 491)
shape of test: (13463, 490)


In [11]:
train_A.head()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,flag,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,U

In [12]:
train_A.describe()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,flag,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,U

In [16]:
train_A.isnull().sum().sort_values(ascending=False)

UserInfo_173    39963
UserInfo_126    39963
UserInfo_48     39951
UserInfo_58     39951
UserInfo_140    39951
                ...  
UserInfo_200     1270
UserInfo_12       409
UserInfo_170      409
flag                0
no                  0
Length: 491, dtype: int64

In [17]:
train_B.head()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,flag,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,U

In [18]:
train_B.describe()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,flag,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,U

In [19]:
train_B.isnull().sum().sort_values(ascending=False)

UserInfo_198       4000
UserInfo_71        4000
UserInfo_185       4000
ProductInfo_118    4000
UserInfo_258       4000
                   ... 
UserInfo_149         10
UserInfo_12           0
UserInfo_170          0
flag                  0
no                    0
Length: 491, dtype: int64

In [20]:
test.head()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,UserIn

In [21]:
test.describe()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,UserIn

In [22]:
test.isnull().sum().sort_values(ascending=False)

UserInfo_173       13461
UserInfo_126       13461
UserInfo_1         13461
UserInfo_198       13459
ProductInfo_118    13453
                   ...  
UserInfo_200          31
UserInfo_149          24
UserInfo_170           0
UserInfo_12            0
no                     0
Length: 490, dtype: int64

In [23]:
train_A.dtypes[train_A.dtypes==np.int64]

no      int64
flag    int64
dtype: object

In [24]:
train_B.dtypes[train_B.dtypes==np.int64]

no              int64
flag            int64
UserInfo_170    int64
dtype: object

In [26]:
train_B['UserInfo_170'].value_counts()

0    4000
Name: UserInfo_170, dtype: int64

In [27]:
train_B = train_B.drop('UserInfo_170',axis=1)

In [28]:
test.dtypes[test.dtypes==np.int64]

no              int64
UserInfo_170    int64
dtype: object

In [29]:
# 严重缺失值处理：将缺失的特征大于99%的特征删除
train_B_info = train_B.describe()

In [30]:
train_B_info.head()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,flag,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,U

In [50]:
useful_col = []
for col in train_B_info.columns:
    if train_B[col].isnull().sum()<train_B.shape[0]*0.99:
        useful_col.append(col)

In [51]:
len(useful_col)

431

In [52]:
len(train_B_info.columns)

490

In [53]:
train_B_1 = train_B[useful_col].copy()

In [54]:
train_B.shape

(4000, 490)

In [55]:
train_B_1.shape

(4000, 431)

In [56]:
# 缺失值填充：用-999
train_B_1 = train_B_1.fillna(-999)

In [57]:
# 选择高线性相关的特征进行删除
relation = train_B_1.corr()

In [58]:
length = relation.shape[0]
final_cols = []
del_cols = []
for i in range(length):
    if relation.columns[i] not in del_cols:
        final_cols.append(relation.columns[i])
        for j in range(i+1,length):
            if (relation.iloc[i,j]>0.98) and (relation.columns[j] not in del_cols):
                del_cols.append(relation.columns[j])

In [59]:
relation

,no,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_19,UserInfo_20,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,ProductInfo_59,UserInfo_72,UserInfo_73,flag,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,ProductInfo_97,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_125,ProductInfo_104,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,UserInfo_135,UserInfo_136,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,UserInfo_144,UserInfo_145,ProductInfo_113,UserInfo_147,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,UserInfo_163,UserInfo_164,ProductInfo_131,UserInfo_165,UserInfo_166,UserInfo_167,ProductInfo_132,ProductInfo_133,UserInfo_168,UserInfo_169,ProductInfo_134,UserInfo_174,ProductInfo_135,ProductInfo_136,ProductInfo_137,ProductInfo_138,UserInfo_175,UserInfo_176,UserInfo_177,ProductInfo_139,ProductInfo_140,UserInfo_178,ProductInfo_141,ProductInfo_142,UserInfo_179,UserInfo_180,ProductInfo_143,WebInfo_3,UserInfo_18

In [60]:
final_cols

['no',
 'UserInfo_2',
 'UserInfo_3',
 'UserInfo_4',
 'ProductInfo_1',
 'UserInfo_5',
 'UserInfo_7',
 'UserInfo_8',
 'UserInfo_11',
 'UserInfo_12',
 'UserInfo_15',
 'UserInfo_16',
 'UserInfo_17',
 'UserInfo_19',
 'UserInfo_20',
 'UserInfo_23',
 'UserInfo_25',
 'UserInfo_27',
 'UserInfo_28',
 'UserInfo_33',
 'UserInfo_36',
 'UserInfo_39',
 'UserInfo_40',
 'UserInfo_44',
 'UserInfo_48',
 'UserInfo_49',
 'UserInfo_52',
 'UserInfo_54',
 'UserInfo_60',
 'UserInfo_63',
 'UserInfo_66',
 'ProductInfo_49',
 'ProductInfo_50',
 'UserInfo_67',
 'UserInfo_68',
 'WebInfo_1',
 'UserInfo_72',
 'UserInfo_73',
 'flag',
 'UserInfo_75',
 'UserInfo_76',
 'UserInfo_80',
 'UserInfo_82',
 'UserInfo_83',
 'UserInfo_85',
 'UserInfo_92',
 'UserInfo_94',
 'UserInfo_96',
 'UserInfo_97',
 'UserInfo_100',
 'UserInfo_101',
 'UserInfo_104',
 'UserInfo_105',
 'UserInfo_106',
 'UserInfo_107',
 'UserInfo_108',
 'ProductInfo_89',
 'UserInfo_113',
 'UserInfo_115',
 'UserInfo_118',
 'UserInfo_121',
 'UserInfo_122',
 'UserInf

In [61]:
train_B_1 = train_B_1[final_cols] 

In [63]:
train_B_flag = train_B_1['flag']
train_B_1.drop('no',axis=1,inplace=True)
train_B_1.drop('flag',axis=1,inplace=True)

In [64]:
import xgboost as xgb

In [65]:
# 模型训练
dtrain_B = xgb.DMatrix(data=train_B_1,label=train_B_flag)

In [71]:
Trate = 0.25
params = {'booster':'gbtree'
         ,'eta':0.1
         ,'max_depth':4
         ,'max_delta_step':0
         ,'subsample':0.9
         ,'colsample_bytree':0.9
         ,'base_score':Trate
         ,'objective':'binary:logistic'
         ,'lambda':5
         ,'alpha':8
         ,'random_state':100
         ,'eval_metric':'auc'}

xgb_model = xgb.train(params,dtrain_B,num_boost_round=200,verbose_eval=True,maximize=True)

In [109]:
prediction = xgb_model.predict(xgb.DMatrix(test[train_B_1.columns].fillna(-999)))
test['flag'] = prediction
# prediction = pd.Series(prediction).apply(swap).values
test.head()

,no,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,ProductInfo_1,UserInfo_5,UserInfo_6,ProductInfo_2,UserInfo_7,ProductInfo_3,ProductInfo_4,UserInfo_8,UserInfo_9,ProductInfo_5,ProductInfo_6,UserInfo_10,UserInfo_11,ProductInfo_7,UserInfo_12,ProductInfo_8,ProductInfo_9,ProductInfo_10,UserInfo_13,ProductInfo_11,ProductInfo_12,UserInfo_14,ProductInfo_13,ProductInfo_14,UserInfo_15,UserInfo_16,UserInfo_17,UserInfo_18,UserInfo_19,UserInfo_20,UserInfo_21,UserInfo_22,UserInfo_23,UserInfo_24,UserInfo_25,ProductInfo_15,ProductInfo_16,ProductInfo_17,UserInfo_26,ProductInfo_18,ProductInfo_19,UserInfo_27,ProductInfo_20,UserInfo_28,ProductInfo_21,ProductInfo_22,ProductInfo_23,UserInfo_29,UserInfo_30,UserInfo_31,UserInfo_32,UserInfo_33,ProductInfo_24,UserInfo_34,ProductInfo_25,ProductInfo_26,ProductInfo_27,ProductInfo_28,ProductInfo_29,ProductInfo_30,ProductInfo_31,UserInfo_35,ProductInfo_32,ProductInfo_33,UserInfo_36,UserInfo_37,ProductInfo_34,UserInfo_38,UserInfo_39,UserInfo_40,ProductInfo_35,ProductInfo_36,UserInfo_41,ProductInfo_37,UserInfo_42,UserInfo_43,UserInfo_44,UserInfo_45,ProductInfo_38,UserInfo_46,UserInfo_47,UserInfo_48,ProductInfo_39,UserInfo_49,UserInfo_50,UserInfo_51,ProductInfo_40,ProductInfo_41,UserInfo_52,ProductInfo_42,ProductInfo_43,ProductInfo_44,UserInfo_53,UserInfo_54,UserInfo_55,UserInfo_56,UserInfo_57,UserInfo_58,ProductInfo_45,UserInfo_59,UserInfo_60,ProductInfo_46,UserInfo_61,UserInfo_62,UserInfo_63,ProductInfo_47,UserInfo_64,UserInfo_65,ProductInfo_48,UserInfo_66,ProductInfo_49,ProductInfo_50,ProductInfo_51,ProductInfo_52,UserInfo_67,UserInfo_68,ProductInfo_53,WebInfo_1,UserInfo_69,ProductInfo_54,ProductInfo_55,ProductInfo_56,ProductInfo_57,ProductInfo_58,UserInfo_70,ProductInfo_59,UserInfo_71,UserInfo_72,UserInfo_73,ProductInfo_60,UserInfo_74,UserInfo_75,UserInfo_76,UserInfo_77,UserInfo_78,ProductInfo_61,UserInfo_79,ProductInfo_62,UserInfo_80,UserInfo_81,UserInfo_82,ProductInfo_63,ProductInfo_64,UserInfo_83,ProductInfo_65,ProductInfo_66,ProductInfo_67,ProductInfo_68,UserInfo_84,UserInfo_85,UserInfo_86,UserInfo_87,UserInfo_88,UserInfo_89,ProductInfo_69,ProductInfo_70,UserInfo_90,UserInfo_91,ProductInfo_71,UserInfo_92,ProductInfo_72,UserInfo_93,UserInfo_94,UserInfo_95,UserInfo_96,UserInfo_97,UserInfo_98,UserInfo_99,UserInfo_100,UserInfo_101,ProductInfo_73,ProductInfo_74,ProductInfo_75,UserInfo_102,ProductInfo_76,UserInfo_103,UserInfo_104,UserInfo_105,ProductInfo_77,UserInfo_106,ProductInfo_78,UserInfo_107,UserInfo_108,UserInfo_109,ProductInfo_79,ProductInfo_80,ProductInfo_81,ProductInfo_82,UserInfo_110,ProductInfo_83,ProductInfo_84,ProductInfo_85,ProductInfo_86,ProductInfo_87,UserInfo_111,UserInfo_112,ProductInfo_88,ProductInfo_89,UserInfo_113,UserInfo_114,ProductInfo_90,ProductInfo_91,ProductInfo_92,ProductInfo_93,UserInfo_115,ProductInfo_94,UserInfo_116,UserInfo_117,UserInfo_118,ProductInfo_95,ProductInfo_96,WebInfo_2,UserInfo_119,ProductInfo_97,UserInfo_120,ProductInfo_98,UserInfo_121,ProductInfo_99,ProductInfo_100,ProductInfo_101,UserInfo_122,ProductInfo_102,UserInfo_123,ProductInfo_103,UserInfo_124,UserInfo_125,ProductInfo_104,UserInfo_126,UserInfo_127,UserInfo_128,UserInfo_129,UserInfo_130,ProductInfo_105,UserInfo_131,UserInfo_132,UserInfo_133,ProductInfo_106,UserInfo_134,ProductInfo_107,UserInfo_135,UserInfo_136,UserInfo_137,UserInfo_138,UserInfo_139,UserInfo_140,ProductInfo_108,UserInfo_141,ProductInfo_109,ProductInfo_110,UserInfo_142,UserInfo_143,ProductInfo_111,ProductInfo_112,UserInfo_144,UserInfo_145,UserInfo_146,ProductInfo_113,UserInfo_147,UserInfo_148,ProductInfo_114,ProductInfo_115,UserInfo_149,UserInfo_150,UserInfo_151,ProductInfo_116,ProductInfo_117,ProductInfo_118,ProductInfo_119,UserInfo_152,UserInfo_153,ProductInfo_120,ProductInfo_121,ProductInfo_122,UserInfo_154,ProductInfo_123,UserInfo_155,ProductInfo_124,UserInfo_156,ProductInfo_125,ProductInfo_126,UserInfo_157,UserInfo_158,ProductInfo_127,ProductInfo_128,ProductInfo_129,ProductInfo_130,UserInfo_159,UserInfo_160,UserInfo_161,UserInfo_162,UserIn

In [110]:
test[['no','flag']].to_csv('submit.csv',index=None)

In [93]:
prediction

array([0.1444398 , 0.07151229, 0.1344157 , ..., 0.07532919, 0.14810285,
       0.06843498], dtype=float32)

In [84]:
from sklearn.metrics import accuracy_score
predict = xgb_model.predict(xgb.DMatrix(train_B_1))
def swap(x):
    if x > 0.5:
        return 1
    else:
        return 0
predict = pd.Series(predict).apply(swap).values
accuracy_score(y_pred=predict,y_true=train_B_flag)

0.92975

In [85]:
predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [86]:
train_B_flag.value_counts()

0    3719
1     281
Name: flag, dtype: int64